In [25]:
import tensorflow as tf
import numpy as np
import scipy as sp
from keras.utils.vis_utils import plot_model
import pydot
from keras_sequential_ascii import keras2ascii
#check imports and versions
for module in [tf, np, sp, pydot]:
    print(module.__name__, module.__version__)
# %pip install plotly.express --upgrade

tensorflow 2.11.0
numpy 1.23.4
scipy 1.9.3
pydot 1.4.2


In [26]:
dataset_path = r'D:\training\data\dataset'

In [27]:
# create a dataset from the images
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='binary',
    class_names=['empty', 'full'],
    color_mode='rgb'
)

Found 999 files belonging to 2 classes.


In [28]:
#print the class names of the dataset
print(dataset.class_names)
#show the first 9 images of the dataset and their labels with cv2

import cv2

for images, labels in dataset.take(1):
    for i in range(9):
        image = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        label = dataset.class_names[int(label)]
        #put the label on the image
        cv2.putText(image, label, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

['empty', 'full']


In [29]:
model = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Flatten(input_shape=(128, 128)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(dataset, epochs=10)



Epoch 1/10
32/32 [==============================] - 9s 258ms/step - loss: 11.6470 - accuracy: 0.6296
Epoch 2/10
32/32 [==============================] - 8s 250ms/step - loss: 1.3863 - accuracy: 0.8839
Epoch 3/10
32/32 [==============================] - 8s 250ms/step - loss: 1.1453 - accuracy: 0.9129
Epoch 4/10
32/32 [==============================] - 8s 249ms/step - loss: 0.8388 - accuracy: 0.9209
Epoch 5/10
32/32 [==============================] - 8s 249ms/step - loss: 0.6524 - accuracy: 0.9309
Epoch 6/10
32/32 [==============================] - 8s 250ms/step - loss: 0.6839 - accuracy: 0.9279
Epoch 7/10
32/32 [==============================] - 8s 255ms/step - loss: 0.4232 - accuracy: 0.9530
Epoch 8/10
32/32 [==============================] - 8s 249ms/step - loss: 0.3364 - accuracy: 0.9600
Epoch 9/10
32/32 [==============================] - 8s 252ms/step - loss: 0.4400 - accuracy: 0.9540
Epoch 10/10
32/32 [==============================] - 8s 249ms/step - loss: 1.7199 - accuracy: 0.888

In [30]:
test_loss, test_acc = model.evaluate(dataset)

print('\nTest accuracy:', test_acc)

32/32 [==============================] - 1s 29ms/step - loss: 0.6533 - accuracy: 0.9540

Test accuracy: 0.9539539813995361


In [31]:
import cv2

for images, labels in dataset.take(1):
    for i in range(9):
        image = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        label = dataset.class_names[int(label)]
        #put the label on the image
        cv2.putText(image, label, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #predict the image
        prediction = model.predict(np.array([image]))
        prediction = dataset.class_names[int(prediction[0][0])]
        #put the prediction on the image
        cv2.putText(image, str(prediction), (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step


In [32]:

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
keras2ascii(model)


           OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

               Input   #####    256  256    3
           Rescaling   ????? -------------------         0     0.0%
                       #####    256  256    3
             Flatten   ||||| -------------------         0     0.0%
                       #####      196608
               Dense   XXXXX -------------------  25165952   100.0%
                relu   #####         128
               Dense   XXXXX -------------------       129     0.0%
             sigmoid   #####           1
